# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto san carlos,-12.6608,-63.5500,302.52,73,96,1.39,BO,1701711539
1,1,papatowai,-46.5619,169.4708,279.86,74,18,1.67,NZ,1701711540
2,2,lihue,21.9770,-159.3688,294.84,86,75,2.06,US,1701711540
3,3,edinburgh of the seven seas,-37.0676,-12.3108,289.48,94,51,8.79,SH,1701711541
4,4,tiksi,71.6366,128.8685,240.52,96,97,3.23,RU,1701711542


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points('Lng', "Lat", geo=True,
                              frame_width = 700, frame_height = 700, color="Humidity")

# Display the map
map1

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
df = city_data_df[city_data_df['Humidity'] < 70]
df = df[df['Wind Speed'] > 8]

# Drop any rows with null values
df = df.dropna(how='any')

# Display sample data
df.sample(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,puerto natales,-51.7262,-72.5060,289.47,51,75,11.32,CL,1701711562
517,517,lewistown,47.0639,-109.4282,278.43,42,0,8.23,US,1701712005
99,99,lebu,-37.6100,-73.6559,287.91,68,0,8.53,CL,1701711627
452,452,rio grande,-53.7857,-67.7016,287.17,58,20,13.89,AR,1701711940
68,68,victorica,-36.2165,-65.4370,295.97,31,41,9.82,AR,1701711595
318,318,kapa'a,-33.9290,18.4174,292.49,49,20,8.75,ZA,1701711322
234,234,stanley,-51.6951,-57.8492,284.04,58,20,10.29,FK,1701711741
342,342,praia,14.9163,-23.5095,301.47,57,20,9.26,CV,1701711845
188,188,vila velha,-20.3297,-40.2920,304.23,66,20,9.26,BR,1701711699
191,191,guarapari,-20.6719,-40.4985,304.22,66,24,8.30,BR,1701711711


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
99,lebu,CL,-37.6100,-73.6559,68,
32,puerto natales,CL,-51.7262,-72.5060,51,
62,punta arenas,CL,-53.1626,-70.9078,62,
318,kapa'a,ZA,-33.9290,18.4174,49,
517,lewistown,US,47.0639,-109.4282,42,
189,sitka,US,57.0525,-135.3376,61,
457,zaragoza,ES,41.6521,-0.8809,50,
234,stanley,FK,-51.6951,-57.8492,58,
191,guarapari,BR,-20.6719,-40.4985,66,
342,praia,CV,14.9163,-23.5095,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 1000 # km
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lon = row.Lat, row.Lng

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Los Pinos
punta arenas - nearest hotel: Hotel Cabo de Hornos
victorica - nearest hotel: Hotel Mazamet
lebu - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
sitka - nearest hotel: Aspen Suites Hotel
guarapari - nearest hotel: Hotel Bom Jesus
stanley - nearest hotel: Shorty's Motel
kapa'a - nearest hotel: The Capital 15 On Orange
dillon - nearest hotel: No hotel found
praia - nearest hotel: Entrada no numero 13
rio grande - nearest hotel: Atlántida Hotel
zaragoza - nearest hotel: Maza
lewistown - nearest hotel: Yogo Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
32,puerto natales,CL,-51.7262,-72.5060,51,Los Pinos
62,punta arenas,CL,-53.1626,-70.9078,62,Hotel Cabo de Hornos
68,victorica,AR,-36.2165,-65.4370,31,Hotel Mazamet
99,lebu,CL,-37.6100,-73.6559,68,No hotel found
188,vila velha,BR,-20.3297,-40.2920,66,Hotel Prainha
189,sitka,US,57.0525,-135.3376,61,Aspen Suites Hotel
191,guarapari,BR,-20.6719,-40.4985,66,Hotel Bom Jesus
234,stanley,FK,-51.6951,-57.8492,58,Shorty's Motel
318,kapa'a,ZA,-33.9290,18.4174,49,The Capital 15 On Orange
331,dillon,US,45.2141,-112.6346,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map1 = hotel_df.hvplot.points('Lng', "Lat", geo=True, hover_cols='all',
                              frame_width = 700, frame_height = 700)

# Display the map
map1

:Points   [Lng,Lat]   (index,City,Country,Humidity,Hotel Name)